# Modelo Hull-White

## Ecuaciones del Modelo

El modelo de Hull - White es una extensión al modelo de Vasicek. La ecuación diferencial estocástica para la evolución de la tasa instantánea en la medida ajustada por riesgo es:

$$
\begin{equation}
dr_t=\left(\theta_t-\gamma^* r_t\right)dt+\sigma dX_t
\end{equation}
$$

La función $\theta_t$, que se puede interpretar como un nivel de reversión a la media variable en el tiempo, se calibra para ajustar perfectamente el valor de mercado de los bonos cupón cero.

Después de eso, quedan aún dos grados de libertad, $\gamma^*$ y $\sigma$, para calibrar la estructura de volatilidades.

### Solución de la Ecuación

La solución a la ecuación diferencial del modelo tiene la siguiente solución:

$$
r_t=r_se^{-\gamma^*\left(t-s\right)}+\int_s^te^{-\gamma^*\left(t-u\right)}\theta_udu+\sigma\int_s^te^{-\gamma^*\left(t-u\right)}dW_u \\
r_t=r_se^{-\gamma^*\left(t-s\right)}+\alpha\left(t\right)-\alpha\left(s\right)e^{-\gamma^*\left(t-s\right)}+\sigma\int_s^te^{-\gamma^*\left(t-u\right)}dW_u 
$$

Donde,

$$
\alpha\left(t\right)=f\left(0,t\right)+\frac{\sigma^2}{2\gamma^{*2}}\left(1-e^{-\gamma^{*} t}\right)^2
$$

Por lo tanto, $r_t$, condicional en $s$, distribuye normal con media y varianza dadas por:

$$
\newcommand{\E}{\mathrm{E}}
\newcommand{\Var}{\mathrm{Var}}
\E_t\left[r_s\right]=r_se^{-\gamma^*\left(t-s\right)}+\alpha\left(t\right)-\alpha\left(s\right)e^{-\gamma^*\left(t-s\right)} \\
\Var_t\left[r_s\right]=\frac{\sigma^2}{2\gamma^*}\left[1-e^{-2\gamma^*\left(t-s\right)}\right]
$$

### Fórmula para $\theta_t$

La función $\theta_t$, que permite obtener los precios de mercado de los bonos cupón cero está dada por:

$$
\begin{equation}
\theta_t=\frac{\partial f\left(0,t\right)}{\partial t}+\gamma^*f\left(0,t\right)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp\left(-2\gamma^* t\right)\right]
\end{equation}
$$

Recordar que:

$$
\begin{equation}
f(t,T)=-\frac{\partial\log P(t,T)}{\partial T} \\
P(t,T)=\exp\left[-r\left(t,T\right)\left(T-t\right)\right]
\end{equation}
$$

por lo tanto,

$$
\begin{equation}
-\log P\left(t,T\right)=r\left(t,T\right)\left(T-t\right) \\
-\frac{\partial \log P\left(t,T\right)}{\partial T}=\frac{\partial r\left(t,T\right) }{\partial T}\left(T-t\right)+r\left(t,T\right)
\end{equation}
$$

finalmente,

$$
\frac{\partial f\left(0,t\right)}{\partial t}=\frac{\partial^2 r\left(0,t\right) }{\partial t^2}t+2\frac{\partial r\left(0,t\right) }{\partial t}
$$

### Fórmulas para Bonos Cero Cupón

La fórmula para la valoración de bonos cero cupón en el modelo de Hull - White es la siguiente:

$$
\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right]
\end{equation}
$$

$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp\left(-\gamma^* (T-t)\right)\right)]
\end{equation}
$$

$$
\begin{equation}
A\left(t,T\right)=\log\frac{Z\left(r_0,0,T\right)}{Z\left(r_0,0,t\right)}+B\left(t,T\right)f\left(0,t\right)-\frac{\sigma^2}{4\gamma^*}B\left(t,T\right)^2\left[1-\exp\left(-2\gamma^* t\right)\right)]
\end{equation}
$$

### Fórmulas para Calls y Puts sobre Bonos Cupón Cero

Son las mismas que para el modelo de Vasicek, para una call:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$


$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{S_Z\left(T_O\right)}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$.

## Implementación del Modelo

In [1]:
from finrisk import QC_Financial_3 as Qcf
from typing import List, Tuple, Callable # Callable es un objeto con operador (). Si c es Callable c(...).
from scipy.interpolate import interp1d
from modules import auxiliary as aux
from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [8]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}', 'rate': '{:.6%}', 't': '{:.4f}'}

Veamos la construcción de la función Theta, la simulación y el cálculo de los factores de descuento con las fórmulas del modelo.

### Función $\theta$

Se obtienen los valores de la curva cero cupón.

In [3]:
curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [72]:
curva.tail().style.format(frmt)

,plazo,tasa,df,t,rate
28,7306,0.9253%,83.092401%,20.0164,0.925324%
29,9133,0.9657%,78.533775%,25.0219,0.965719%
30,10957,0.9881%,74.332619%,30.0192,0.988103%
31,14610,0.9408%,68.619685%,40.0274,0.940832%
32,18262,0.8464%,65.475678%,50.0329,0.846426%


Por comodidad reescribamos la fórmula para la función $\theta_{t}$:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

Necesitamos una representación en tiempo contínuo de la curva para poder calcular $f(0,t)$ y su derivada. Para eso, vamos a utilizar el objeto `interp1d` de `scipy.interpolate` que permite definir las abcisas de la curva como $t\in\mathbb{R}$.

In [10]:
curva['t'] = curva['plazo'] / 365.0

In [11]:
curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)

In [73]:
curva.tail(20).style.format(frmt)

,plazo,tasa,df,t,rate
13,306,0.0714%,99.940196%,0.8384,0.071356%
14,335,0.0708%,99.934996%,0.9178,0.070848%
15,365,0.0702%,99.929787%,1.0000,0.070238%
16,547,0.0642%,99.903893%,1.4986,0.064161%
17,730,0.0568%,99.886549%,2.0000,0.056758%
18,1097,0.0750%,99.774813%,3.0055,0.075010%
19,1462,0.1258%,99.497569%,4.0055,0.125752%
20,1826,0.1939%,99.034837%,5.0027,0.193864%
21,2191,0.2775%,98.348253%,6.0027,0.277463%
22,2556,0.3603%,97.508202%,7.0027,0.360340%


A continuación, se construye el objeto:

In [13]:
zcurva = interp1d(
    curva['t'],
    curva['rate'],
    kind='cubic',
    fill_value="extrapolate"
)

In [16]:
zcurva(1)

array(0.00070238)

Veamos un gráfico:

In [23]:
steps_per_year = 90
years = 20
dt = 1 / steps_per_year
result = []
for i in range(0, steps_per_year * years):
    result.append((i * dt, zcurva(i * dt)))
df_result = pd.DataFrame(result, columns=['plazo', 'tasa'])

In [24]:
fig = px.line(
    df_result,
    x='plazo',
    y='tasa',
    title=f'Curva Interpolada'
)
fig.show()

Se definen 3 funciones `float -> float` para calcular el valor de la tasa, su derivada y su derivada segunda en un tiempo $t>0$. Las dos derivadas se calculan desde la izquierda.

In [26]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t) - zrate(t - delta)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t) - dzrate(t - delta)) / delta

In [27]:
t = 2
print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')

zrate(2.00) = 0.0568%
dzrate(2.00) = -0.0066%
d2zrate(2.00) = 0.0584%


Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$,

y también que:

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

Podemos ahora definir las funciones `fwd(0,t)` y `dfwd(0,t)`. Notar que las definiciones para `zrate`, `dzrate` y `d2zrate` se capturan del contexto global (que en este caso es el notebook).

In [28]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [16]:
print(f'fwd({t:.2f}) = {fwd(t):.4%}')
print(f'dfwd({t:.2f}) = {dfwd(t):.4%}')

fwd(2.00) = 0.0436%
dfwd(2.00) = 0.1036%


Con esto, podemos ya escribir una expresión para la función Theta. Definamos un valor para $\sigma$ y $\gamma^*$:

In [38]:
sigma = .015
gamma = .5

# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(0)

In [39]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [40]:
print(f'theta({t:.2f}) = {theta(t):.4%}')

theta(2.00) = 0.1448%


In [41]:
def func(a, b, c):
    return a * b * c

In [46]:
def fix_a_b(a, b):
    def wrap(c):
        return func(a, b, c)
    return wrap

In [47]:
fixed12 = fix_a_b(1, 2)

In [48]:
fixed12(3)

6

Grafiquemos la función:

In [49]:
thetas = []
dt = 1 / 365.0
for i in range(0, 8000):
    t = i * dt
    thetas.append((i * dt, theta(t)))
df_theta = pd.DataFrame(thetas, columns=['t', 'theta'])

In [50]:
fig = px.line(
    df_theta,
    x='t',
    y='theta',
    title=f'Función theta(t)'
)
fig.show()

### Simulación

Finalmente, podemos simular.

In [51]:
def sim_hw(
    gamma: float,
    sigma: float,
    theta: Callable[[float, ], float],
    r0: float,
    num_steps: int,
    seed=None
) -> List[Tuple[float, float]]:
    r = r0
    dt = 1 / 264.0 # Proxy del número de días hábiles en 1Y
    sqdt = math.sqrt(dt)
    sim = [(0, r0)]
    rnd.seed(seed)
    for i in range(1, num_steps + 1):
        # print(str(r))
        epsilon = rnd.normal()
        r = r + (theta((i - 1) * dt) - gamma * r) * dt + sigma * sqdt * epsilon # Esquema de Euler
        sim.append((i * dt, r))
    return sim

In [60]:
sim = sim_hw(gamma, sigma, theta, r0, 528, 1234)

In [61]:
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])

In [62]:
df_sim.tail()

,t,tasa
524,1.984848,0.00314848
525,1.988636,0.00226927
526,1.992424,0.00215287
527,1.996212,0.00218084
528,2.000000,0.00412458


In [63]:
fig = px.line(
    df_sim,
    x='t',
    y='tasa',
    title=f'Simulación Hull-White'
)
fig.show()

In [66]:
que_step = 264
print(f'plazo: {sim[que_step][0]:.4f},\ntasa: {sim[que_step][1]:.4%}')

plazo: 1.0000,
tasa: 0.9285%


### Bono Cupón Cero

Función $B$ de la fórmula para un bono cupón cero:

In [69]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

Función $A$ de la fórmula para un bono cupón cero:

In [68]:
def a_hw(
    zrate: float,
    fwd, gamma: float,
    sigma: float,
    t: float,
    T: float,
    verbose=False
) -> float:
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

Factor de descuento según HW.

In [70]:
def zero_hw(
    r: float,
    gamma: float,
    sigma: float,
    zrate: float,
    fwd: float,
    t: float,
    T: float
) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

Verifiquemos que los valores que entrega la fórmula de HW coinciden con los datos de la curva:

In [75]:
r0 = float(zrate(0))
print(f"r0:        {r0:.8%}")
t = 0
T = 9.1234
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print(f"df hw:    {z:.8%}")
print(f"df curva: {math.exp(-zrate(T) * T):.8%}")

r0:        0.07881405%
df hw:    95.36685521%
df curva: 95.36685521%


### Ejercicio de Valorización

Vamos a valorizar, en 1Y más, un bono semestral a tasa fija del 5% que en 1Y más le quedarán 4Y. Podemos además hacer el ejercicio de verificar la simulación calculando los valores de los bonos cero cupón usando la simulación.

Utilizaremos la siguiente función:

In [76]:
def bono_tasa_fija(
    start_time: float,
    yf: float,
    num_cupones: int,
    valor_tasa: float
) -> List[Tuple[float, float]]:
    """
    Retorna los plazos y flujos de un bono a tasa fija bullet con nominal = 100.

    params:

    - start_time: fecha (expresada en fracción de año) en que comienza el devengo del primer cupón.
    - yf: fracción de año que representa la periodicidad del bono (yf = .5 -> bono semestral).
    - num_cupones: número de cupones del bono
    - valor_tasa: valor de la tasa fija del bono. Los intereses se calculan de forma lineal.

    return:

    - Una `list` de `tuple` con la fecha de pago del cupón (como instante de tiempo) y el monto del cupón.
    """
    result = []
    nominal = 100.0
    flujo = nominal * valor_tasa * yf
    for i in range(1, num_cupones + 1):
        if i == num_cupones:
            flujo += nominal
        result.append((i * yf + start_time, flujo))
    return result

In [78]:
start_time = 1
yf = .5
tasa = .05
num_cupones = 8
bf = bono_tasa_fija(start_time, yf, num_cupones, tasa)
bf

[(1.5, 2.5),
 (2.0, 2.5),
 (2.5, 2.5),
 (3.0, 2.5),
 (3.5, 2.5),
 (4.0, 2.5),
 (4.5, 2.5),
 (5.0, 102.5)]

In [86]:
sim = sim_hw(gamma, sigma, theta, r0, 264, 1001)

In [87]:
r1Y = sim[264][1]
print(f'Tasa instantánea simulada a 1Y: {r1Y:.6%}')

Tasa instantánea simulada a 1Y: 0.027686%


¿Cuál es la curva de factores de descuento en t = 1Y en esta simulación?

In [91]:
t = 1
zeros_1Y = []
for plazo in [b[0] for b in bf]:
    zeros_1Y.append(zero_hw(r1Y, gamma, sigma, zrate, fwd, t, plazo))
for p, z in zip([b[0] for b in bf], zeros_1Y):
    print(f'Df al plazo {p}Y: {z:.6%}')

Df al plazo 1.5Y: 99.987873%
Df al plazo 2.0Y: 99.979417%
Df al plazo 2.5Y: 99.948169%
Df al plazo 3.0Y: 99.878076%
Df al plazo 3.5Y: 99.764499%
Df al plazo 4.0Y: 99.605664%
Df al plazo 4.5Y: 99.401232%
Df al plazo 5.0Y: 99.143943%


¿Cuál es el valor presente?

In [92]:
vp = np.dot(zeros_1Y, [b[1] for b in bf])
print(f"vp: {vp:.6f}")

vp: 119.086665


Chequear que el operador `np.dot` haga lo que creemos que está haciendo:

In [93]:
vp1 = 0
for y in zip([b[1] for b in bf], zeros_1Y):
    vp1 += y[0] * y[1]
print(f"vp1: {vp1:.6f}")

vp1: 119.086665


### Simulación con Muchas Trayectorias

Para hacer la valorización anterior muchas veces, es necesario generar muchas simulaciones distintas. Podríamos proceder de la siguiente forma:

In [102]:
num_sim = 10
def sim_hw_many_v1(
    num_sim: int,
    r0: float,
    gamma: float,
    sigma: float,
    zrate,
    fwd,
    theta,
    bf
):
    result = []
    # seeds = [1,2,3,4,5,6,7,8,9,10]
    for i in range(0, num_sim):
        r1y = sim_hw(gamma, sigma, theta, r0, 264, seed=None)[263][1]
        zeros_1Y = []
        for plazo in [b[0] for b in bf]:
            zeros_1Y.append(zero_hw(r1y, gamma, sigma, zrate, fwd, t, plazo))
        vp = np.dot(zeros_1Y, [b[1] for b in bf])
        result.append(vp)
    return result


res = sim_hw_many_v1(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf)
res

[117.56869205390966,
 123.14123996316098,
 124.2360947654676,
 117.7663241468414,
 117.6519014060664,
 120.74690425895037,
 119.12085111218319,
 118.7300211288645,
 118.48663985380924,
 119.47185215749415]

Los 10 valores de arriba son el valor presente en **t = 1Y**, de los 8 cupones restantes del bono.

Podríamos utilizar un algortimo de este tipo para valorizar cualquier payoff, que dependa de la tasa corta, en el futuro. Sin embargo, falta un paso fundamental, traer a valor presente. Recordemos las siguientes ecuaciones (notebooks 4 y 5):

$$
\begin{equation}
\Pi\left(t,X\right)=\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\Pi\left(T,X\right)\right]
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)=\exp\left[-\int_t^Tr_c\left(u\right)du\right]
\end{equation}
$$

$$
\begin{equation}
\mathbb{E}_t^{Q_f}\left[D_c\left(t,T\right)\right]=P_c\left(t,T\right)
\end{equation}
$$

$$
\begin{equation}
D_c\left(t,T\right)={B_c\left(t,T\right)}^{-1}
\end{equation}
$$

Para fijar las ideas, supongamos que el payoff depende de la curva de colateral y calculemos el factor de descuento por simulación.

E(valor_presente) = E(exp(-Int(0,T)r(s)ds) x Payoff(rT, T))

In [109]:
num_sim = 10
sim = 0
num_steps = 263
print(f"num_steps: {num_steps}")
dt = 1 / 264.0
# result = []
for i in range(0, num_sim):
    r_sim = [x[1] for x in sim_hw(gamma, sigma, theta, r0, num_steps, seed=None)]
    # print(type(r_sim))
    # exp(Suma(i)(-dt*ri)) = Prod(i)exp(-dt * ri)
    temp = math.exp(np.sum(- dt * np.array(r_sim)))
    # result.append(temp)
    sim += temp

# print(result)
ez = sim / num_sim
z_curva = math.exp(-zrate(t) * t)
print(f"ez: {ez:.8f}")
print(f"z_curva: {z_curva: .8f}")

num_steps: 263
ez: 0.99884805
z_curva:  0.99929787


Una de las desventajas de ese approach es que la simulación es difícil de reproducir porque la semilla cambia en cada iteración.

In [110]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

In [123]:
num_sim = 1000
num_steps = 263
print(f"num_steps: {num_steps}")
seed = 1234
df = 0
tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    df += math.exp(-dt * np.sum(sim))

ez = df / num_sim
z_curva = math.exp(-zrate(t) * t)
print(f"ez: {ez: .8%}")
print(f"z_curva: {z_curva: .8%}")

num_steps: 263
ez:  99.95341501%
z_curva:  99.92978688%


In [41]:
tiempo, sim = sim_hw_many(gamma, sigma, theta, r0, 10, 264, 1000)